Welcome in this notebook. Instruction will be given during the cells the implement the problem correctly.

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "20"
os.environ["MKL_NUM_THREADS"] = "20"
os.environ["NUMEXPR_NUM_THREADS"] = "20"
from importlib import reload
import yee_uchie_QM_pml_class
reload(yee_uchie_QM_pml_class)
from yee_uchie_QM_pml_class import Source, Recorder, QM_UCHIE_params, Yee_UCHIE
import QM_update as QM
import numpy as np
import scipy.constants as ct
import time
from scipy.special import hankel2
import PyQt5
import matplotlib
matplotlib.use('Qt5Agg')  # Make sure to set the backend before importing pyplot

import matplotlib.pyplot as plt
%matplotlib qt

 # Make sure to set the backend before importing pyplot





In [2]:
eps0 = ct.epsilon_0
mu0 = ct.mu_0
hbar = ct.hbar #J⋅s
m = ct.electron_mass*0.15
q = -ct.elementary_charge
c0 = ct.speed_of_light 


Z0 = np.sqrt(mu0/eps0)


Set the parameters for the FDTD-UCHIE subgridding scheme coupled with the QM system. The meaning of the paremeters are explained as comments next to it.

The subgridding is done by uniformaly fine gridding the coarse where the cells are evenly seperated across the coarse grid. So e.g. if n=5 and N_sub=15, this will take 3 coarse grids and fine grid each of them with 5. So in total 3 coarse grids will be fine gridded giving you a total 5*15=75 cells in the x-direction across 3 coarse grid. So the discretisation step of the fine grid dx_f = dx/(N_sub*n).

In [3]:
## Define parameters of the coarse grid/FDTD region ##

Nx = 400 # Choose the number of coarse grid in the x-direction of your FDTD region (before subgridding)
Ny = 400 # Choose the number of coarse grid in the y-direction of your FDTD region (before subgridding)
Nt = 100000 # Choose the number of steps to be performed

dx = 0.25e-10 # x-direction's discretisation step of the coarse grid [m]
dy = 0.25e-10 # y-direction's discretisation step of the coarse grid [m]
courant = 0.9 # Courant number, this number should be chosen smaller than 1
dt = courant * 1/(np.sqrt(1/dx**2 + 1/dy**2)*ct.c) # discretisation of the time step [s]

## parameters of the finegrid/UCHIE region

Ly = 3/5*Ny*dy # Height (y-direction) of your UCHIE region
n = 5 # The number of fine gridding in one coarse grid
N_sub = 15 # The numbers of coarse grid that you want to fine grid

x_sub1 = Nx//3*dx # The location where you want the first subgridding to take place (so left interface of the UCHIE/subgridding region)
x_sub2 = 2*Nx//3*dx # The location where you want the second subgridding to take place (so left interface of the UCHIE/subgridding region)



## Parameters of the QM wire ##

QMxpos1 = n*N_sub//2 # The location of the first QM-wire, now it is set in the middle of the UCHIE scheme
QMxpos2 = n*N_sub//2 # The location of the second QM-wire, now it is set in the middle of the UCHIE scheme

NyQM = int(2*Ny/5) # Length of the QM wires

N = 1e24 # particles/m2 in the QM wire
order = 'fourth' # The order of QM scheme, set this to either 'second' or 'fourth'
omega = 50e14 # [rad/s]
alpha = 0

## create the Gaussian magnetic current source (see definition of the gaussian) ##
xs = 1/4*Nx*dx # x-location of the magnetic current source
ys = 3*Ny/5*dy # y-location of the magnetic current source
tc = dt*Nt/30 
sigma = tc/6
J0 = 1e2/dx/dy
source = Source(xs, ys, J0, tc, sigma) # Creating the source as a Class


## Parameters for the PML see definition of the PML parameters too ##
pml_nl = 20 # Numbers of PML layers
pml_m = 4



Now the important Class objects will be created. Only the objects of the class Recorder can be changed to your choice. The first argument of Recorder takes the x-position of the recorders location and the second argument the y-position.

In [ ]:
potential = QM.Potential(m,omega, NyQM, dy) # The Harmonic potential in your QM wire

# Create the QM scheme for both QM-wire
QMscheme1 = QM.QM(order,NyQM,dy, dt, hbar, m, q, alpha, potential, omega, N)
QMscheme2 = QM.QM(order,NyQM,dy, dt, hbar, m, q, alpha, potential, omega, N)

#Create the recorders
recorder1 = Recorder(0.74*Nx*dx, 0.5*Ny*dy)
recorder2 = Recorder(0.40*Nx*dx, 0.5*Ny*dy)
recorders = [recorder1, recorder2]

params = [QM_UCHIE_params(Ly, n, N_sub, x_sub1, QMxpos1, QMscheme1), QM_UCHIE_params(Ly, n, N_sub, x_sub2, QMxpos2, QMscheme2 )]

The coupled problem will start running here, so the Yee-uchie scheme will be used. From here one, the user should just run the cells in order to get the plots of the coupled system.

In [4]:
start_time = time.time()
problem = Yee_UCHIE(Nx, Ny, Nt, dx, dy, dt, [source], pml_nl, pml_m, qm_uchie_params = params, recorders=recorders, coupled = True)
problem.calculate_fields()
end_time = time.time()
print("Execution time:", end_time - start_time, "seconds")

 87%|████████▋ | 86756/100000 [06:04<00:51, 256.07it/s]

In [ ]:
%matplotlib qt
fig, ax = plt.subplots()

ax.set_xlabel('Time [s]')
ax.set_ylabel('$H_{z}$ [A/m]')

# Plot the main data
ax.plot(recorder1.data_time, np.array(recorder2.data)/mu0, label = 'recorder after 1st subgridding')
ax.plot(recorder2.data_time, np.array(recorder1.data)/mu0, label = 'recorder after 2nd subgridding')
ax.legend()
plt.show()


In [ ]:
%matplotlib qt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.ticker as ticker

fig, ax = plt.subplots()

ax.set_xlabel('Time [s]')
ax.set_ylabel('$H_{z}$ [A/m]')

# Plot the main data
ax.plot(recorder1.data_time, np.array(recorder2.data)/mu0, label = 'recorder after 1st subgridding')
ax.plot(recorder2.data_time, np.array(recorder1.data)/mu0, label = 'recorder after 2nd subgridding')
ax.legend()

# Create an inset axes object with a size of 30% of the parent axes in both dimensions
# Adjust the bbox_to_anchor values to position the inset a bit higher than the bottom right corner
axins = inset_axes(ax, width="69.05%", height="30%", loc=3, bbox_to_anchor=(0.209, 0.1, 1, 1), bbox_transform=ax.transAxes)

# Plot the same data in the inset, but with different x and y limits
axins.plot(recorder1.data_time, np.array(recorder2.data)/mu0)
axins.plot(recorder2.data_time, np.array(recorder1.data)/mu0)
axins.set_xlim([1e-15, 5e-15])  # x limits
axins.set_ylim([-100000, 100000])  # y limits

axins.xaxis.set_major_locator(ticker.MultipleLocator(base=1e-15))

# Create a dashed rectangle around the x and y limits of the inset
mark_inset(ax, axins, loc1=2, loc2=1, fc="none", ec="0.5")

plt.show()


In [ ]:
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 75
plt.ioff()
v = np.max(problem.data_yee)*0.5
anim = problem.animate_field(v)
from IPython.display import HTML
HTML(anim.to_jshtml())


In [ ]:
%matplotlib inline

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 75
plt.ioff()
v = np.max(problem.data_yee[200:])*1.2
anim = problem.animate_field(v)
HTML(anim.to_jshtml())


In [ ]:
%matplotlib inline
plt.plot(problem.recorders[0].data)


In [ ]:
%matplotlib inline
QMscheme1.expvalues('energy')



In [ ]:
QMscheme2.expvalues('energy')


In [ ]:
QMscheme1.heatmap()
